# 与活动交互<a class="anchor" id="top"></a>

在本笔记本中，您将在 Amazon Personalize 中部署活动并与之交互。

1. [简介](#intro)
1. [创建活动](#create)
1. [与活动交互](#interact)
1. [批量推荐](#batch)
1. [总结](#wrapup)

## 简介<a class="anchor" id="intro"></a>
[返回页首](#top)

此时，您应该有多个解决方案，每个解决方案至少有一个解决方案版本。一旦创建了解决方案版本，就可以从这些版本中获得推荐，并了解整体表现。

本笔记本首先将以前笔记本中的每个解决方案版本部署到各个活动中。一旦它们处于活跃状态，就有用于查询推荐的资源，并有辅助函数将输出内容转化为更易于阅读的内容。

与 Amazon Personalize 上您的客户一样，您可以修改辅助函数以适应数据输入文件的结构，从而使附加呈现工作正常进行。

要开始工作，我们需要再次导入库，加载以前笔记本中的值，并加载 SDK。

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## 与活动交互<a class="anchor" id="interact"></a>
[返回页首](#top)

现在，所有活动都已部署并处于活跃状态，我们可以开始通过 API 调用获取推荐。每个活动都基于不同的配方，活动的行为方式略有不同，因为它们服务于不同的用例。我们将以不同于以前笔记本中使用的顺序介绍每个活动，以便按升序处理可能的复杂活动（即先处理最简单的活动）。

首先，让我们创建一个支持函数来帮助理解 Personalize 活动返回的结果。Personalize 仅返回一个 `item_id`。这对于保持数据紧凑非常有用，但这意味着您需要查询数据库或查找表，才能获得笔记本的用户可读结果。我们将创建一个 helper 函数，用于从 LastFM 数据集返回用户可读的结果。

首先加载数据集，我们可以将数据集用于我们的查找表。

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

通过将 ID 列定义为索引列，只需查询 ID 即可返回艺术家。

In [ ]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

这并不可怕，但在我们的代码中到处重复这一点会变得很混乱，所以下面的函数将对查询进行清理。

In [ ]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

现在，让我们测试几个简单的值来检查我们的错误捕获。

In [ ]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

棒极了！现在我们有了一种呈现结果的方法。 

### SIMS

SIMS 只需要将一个项目作为输入，它将返回用户与之交互的项目，方式与输入项目交互的方式类似。在此特定案例中，项目是一部电影。 

下面的单元格将处理从 SIMS 获取推荐事宜和呈现结果。让我们看看我们之前在本笔记本中查看的第一个项目的推荐是什么（《终结者 2：审判日》）。

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [ ]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

恭喜，这是您的第一个推荐列表！这份列表非常棒，但最好能在一个不错的数据框中显示我们的艺术家样本集的推荐。我们再创建一个辅助函数来实现这一点。

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

现在，让我们用几部不同的电影来测试辅助函数。让我们从数据集中抽取一些数据样本，以测试我们的 SIMS 活动。从我们的数据框中随机获取 5 部电影。

注意：我们将显示类似的标题，因此您可能需要重新运行该样本，直到您识别出列表中的一些电影

In [ ]:
samples = items_df.sample(5)
samples

In [ ]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

您可能会注意到许多项目看起来相同，希望并非所有项目都相同（这更可能是因为交互次数较少，这在 Movielen 小数据集中更常见）。这表明在评估解决方案版本时，评估指标不应是您唯一依赖的指标。因此，当发生这种情况时，您可以采取什么措施来改善该结果？

此时非常适合考虑使用 Personalize 配方的超参数。SIMS 配方具有 `popularity_discount_factor` 超参数（请参阅 [文档](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)）。您可以利用此超参数控制结果中看到的细微差别。此参数及其行为对于您遇到的每个数据集都是唯一的，并且取决于业务目标。您可以迭代此超参数的值，直至获得满意结果，或者您可以通过利用 Personalize 的超参数优化（HPO）功能开始。有关超参数和 HPO 调整的更多信息，请参阅[文档](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html)。

### 用户个性化

HRNN 是 Amazon Personalize 提供的更高级算法之一。该算法支持根据特定用户过去的行为对项目进行个性化，并且可以接收实时事件，以便在无需再训练的情况下更改对用户的推荐。

由于 HRNN 依赖于用户抽样，因此我们加载所需的数据，并选择 3 个随机用户。由于 Movielens 不包括用户数据，我们将从数据集的用户 ID 范围中选择 3 个随机数字。

In [ ]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

现在，我们为上面的 3 个随机用户提供推荐。在此之后，我们将探索实时交互，然后再进行个性化排名。

同样，我们还要创建辅助函数，在一个很好的数据框中呈现结果。

#### API 调用结果

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

在这里，我们清楚地看到，针对每个用户的推荐是不同的。如果需要缓存这些结果，您可以首先通过所有用户运行 API 调用并存储结果，也可以使用批量导出，这将在此笔记本的后面介绍。

现在，让我们应用项目筛选器，来查看针对某个流派中的其中一个用户的推荐


In [ ]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

您可以查看给定流派的电影推荐。在 VOD 应用程序中，您可以使用这些筛选器轻松创建搁板（也称为轨道或传送带）。根据您拥有的项目相关信息，您还可以筛选其他信息，如关键词、年份/年代等。

In [ ]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

下一个主题是实时事件。Personalize 能够侦听来自应用程序的事件，以便更新显示给用户的推荐。这在媒体工作负载（如视频点播）中尤其有用，在这种情况下，客户的意图可能会根据他们是与孩子一起观看还是独自观看而有所不同。

此外，通过此系统记录的事件将被存储，直到您删除调用，并且在您训练下个模型时，事件将与您提供的其他交互数据一起用作历史数据。

#### 实时事件

首先创建一个附加到活动的事件跟踪器。

In [ ]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

我们将创建一些代码来模拟用户与特定项目的交互。运行此代码后，您将获得与上述结果不同的推荐。

我们首先创建一些模拟实时事件的方法。

In [ ]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

此时，我们还没有生成任何实时事件；我们只设置了代码。为了比较实时事件之前和之后的推荐，让我们选择一个用户并为该用户生成原始推荐。

In [ ]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

现在，在应用任何实时事件之前，我们有了针对此用户的推荐列表。现在，让我们选择 3 个随机艺术家，我们将模拟我们的用户交互，然后看看这会如何改变推荐。

In [ ]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [ ]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

现在，我们可以看看点击事件如何改变推荐。

In [ ]:
user_recommendations_df

在以上单元格中，索引后的第一列是用户个性化的默认推荐，该列之后的每一列都有一个通过实时事件与之交互的艺术家的标题，以及此事件发生后的推荐。

行为可能不会发生很大变化；这是由于该数据集的性质相对有限，以及受到几次随机点击的影响。如果您想更好地了解这一点，请尝试模拟一下点击更多电影，您应该会看到更明显的影响。

现在让我们来看看事件筛选器，它支持您根据交互数据筛选项目。对于此数据集，可以根据我们导入的数据点击或观看，但也可以基于您设计的任何交互模式（点击、评级、点赞、观看、购买等）。对于 VOD 搁板，您可以将标题从"最适合您的选择"移至"再次观看"，“再次观看”推荐将基于用户当前的交互，但仅推荐已观看的标题。


In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

现在，让我们发送一个观看事件，以获得 4 条未观看的推荐，这将模拟观看 4 部电影。在 VOD 应用程序中，您可以选择在观看大部分（超过 75%）内容后发送事件。在 100% 完成时发送可能会遗漏那些还缺少积分就停止的人员。

In [ ]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

现在，我们可以看看事件筛选器，以查看更新的已观看和未观看的推荐 

In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

### 个性化排名

个性化排名的核心用例是收集项目，并按照用户兴趣的优先顺序或可能的顺序呈现项目。对于 VOD 应用程序，您希望根据某些信息（导演、位置、超级英雄系列、电影时间段等）动态呈现个性化的搁板/轨道/传送带。这可能并非您的元数据中的信息，因此项目元数据筛选器将不起作用，但您可能在系统中拥有此信息以生成项目列表。

为了证明这一点，我们将使用之前的相同用户和随机的项目集合。

In [ ]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

现在，构建一个显示输入数据的良好数据框。

In [ ]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

然后，进行个性化排名 API 调用。

In [ ]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

现在，将重新排序的项目作为第二列添加到原始数据框中，以便进行并排比较。

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

您可以在上面看到每个条目是如何根据模型对用户的理解重新排序的。当您有项目集合以显示用户时，这是一项常用任务，例如促销列表。

## 批量推荐<a class="anchor" id="batch"></a>
[返回页首](#top)

在许多情况下，您可能希望拥有更大的已导出推荐数据集。最近，Amazon Personalize 推出了批量推荐，作为将推荐集合导出到 S3 的一种方式。在此示例中，我们将介绍如何为 HRNN 解决方案执行此操作。有关批量推荐的更多信息，请参阅[文档](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch)。此功能适用于所有配方，但输出格式会有所不同。

一个简单的实现方法如下所示：

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

SDK 导入、解决方案版本 ARN 和角色 ARN 均已确定。这只剩下输入、输出和待定义的作业名称。

从 HRNN 的输入开始，显示如下：


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

这将产生如下所示的输出：

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

输出是一个 JSON 行文件。该文件由单独的 JSON 对象组成，每行一个对象。因此，我们稍后需要投入更多的工作来转化这种格式的结果。

### 构建输入文件

使用批量功能时，可以指定要在作业完成时接收推荐的用户。下面的单元格将再次选择几个随机用户，然后构建文件并将文件保存到磁盘。从那里，您将把它上载到 S3，以便稍后在 API 调用中使用。

In [ ]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

将文件上载到 S3，并将路径保存为变量以备后用。

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

批量推荐从我们上载到 S3 的文件中读取输入。类似地，批量推荐将输出保存到 S3 中的文件。因此，我们定义了应用于保存结果的输出路径。

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

现在，只需进行调用以启动批量导出过程。

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

运行下面的 While 循环以跟踪批量推荐调用的状态。这个过程可能需要大约 30 分钟才能完成，因为 Personalize 需要支持基础结构来执行任务。我们正在使用仅有 3 个用户的数据集测试该功能，这不是对该机制的有效使用。通常，您只会将此功能用于批量处理，在这种情况下，效率将变得非常明显。

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## 总结<a class="anchor" id="wrapup"></a>
[返回页首](#top)

现在，您拥有了一套完整的模型集合，可处理各种推荐和个性化方案，以及操控客户数据的技能以更好地与服务集成，并了解如何通过 API 和利用开放源代码数据科学工具完成所有这些任务。

使用这些笔记本作为客户 POC 入门指南。当您发现组件缺失或发现新方法时，请提交一个拉取请求，并提供此集合中可能缺少的任何其他有用的组件。

您需要确保清理在此 POC 期间部署的所有资源。我们提供了一个单独的笔记本，向您展示如何识别和删除 `06_Clean_Up_Resources.ipynb` 中的资源。

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn